In [ ]:
#This script is designed to merge raw GoT datafiles into an SQL database containing all relevant 
#information (character name, parentage, etc.) required for production of family tree depictions.

import sqlite3
import json

In [ ]:
#First we'll initialize the database connection and create the needed tables.
conn = sqlite3.connect('got_treedata.db')
c=conn.cursor()
c.execute('''CREATE TABLE PEOPLE
             (ID int primary key, Name text, Gender int, YOB int, YOD int, House text)''')

c.execute('''CREATE TABLE PARENTAGE
             (Parent text, Child text, PRIMARY KEY(Parent,Child))''')



In [ ]:
#Now we'll populate the PEOPLE table with relevant demographic information.
import csv
addrows=[]
with open('dataraw/character-predictions.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count > 0:
            #addrows.append((row[0],row[5],row[9],row[10],row[11],row[12],row[14]))
            addrows.append((row[0],row[5],row[7],row[9],row[10],row[14]))
            #set_trace()      
            #print('Attributes')
        line_count += 1
    print(f'Processed {line_count} lines.')
    
c.executemany('INSERT INTO PEOPLE VALUES (?,?,?,?,?,?)', addrows)    

In [ ]:
#We do the same in turn for the characters database -- this time to connect parents to children.
with open('dataraw/characters.json') as json_file:
    data_relationships = json.load(json_file)['characters']    
for char_record in data_relationships:
    if 'parents' in char_record: 
        for par_name in char_record['parents']:        
            c.execute('INSERT INTO PARENTAGE VALUES (?,?)',(par_name,char_record['characterName']))

In [ ]:
#Commit our changes and close up shop.
conn.commit()
conn.close()